In [3]:
import numpy as np
import pandas as pd
import re

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import sent_tokenize,word_tokenize

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ayush\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
data = pd.read_csv('netflix_titles.csv')
data.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [5]:
data.groupby('type').count()

,show_id,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
type,,,,,,,,,,,
Movie,6131,6131,5943,5656,5691,6131,6131,6129,6128,6131,6131
TV Show,2676,2676,230,2326,2285,2666,2676,2674,2676,2676,2676


In [6]:
data = data.dropna(subset=['cast', 'country', 'rating'])

In [7]:
movies_des = data[data['type'] == 'Movie'].reset_index()
movies_des = movies_des[['title', 'description']]
movies_des.head()

,title,description
0,Sankofa,"On a photo shoot in Ghana, an American model s..."
1,The Starling,A woman adjusting to life after a loss contend...
2,Je Suis Karl,After most of her family is murdered in a terr...
3,Jeans,When the father of the man she loves insists t...
4,Grown Ups,Mourning the loss of their beloved junior high...


In [8]:
tv_des = data[data['type'] == 'TV Show'].reset_index()
tv_des = tv_des[['title', 'description']]
tv_des.head()

,title,description
0,Blood & Water,"After crossing paths at a party, a Cape Town t..."
1,Kota Factory,In a city of coaching centers known to train I...
2,The Great British Baking Show,A talented batch of amateur bakers face off in...
3,Dear White People,Students of color navigate the daily slights a...
4,Falsa identidad,Strangers Diego and Isabel flee their home in ...


In [9]:
filtered_movies = []
movies_words = []
for text in movies_des['description']:
    text_tokens = word_tokenize(text)
    tokens_without_sw = [word.lower() for word in text_tokens if not word in stopwords.words()]
    movies_words.append(tokens_without_sw)
    filtered = (" ").join(tokens_without_sw)
    filtered_movies.append(filtered)

movies_words = [val for sublist in movies_words for val in sublist]
movies_words = sorted(set(movies_words))
movies_des['description_filtered'] = filtered_movies
movies_des.head()

,title,description,description_filtered
0,Sankofa,"On a photo shoot in Ghana, an American model s...","on photo shoot ghana , american model slips ba..."
1,The Starling,A woman adjusting to life after a loss contend...,a woman adjusting life loss contends feisty bi...
2,Je Suis Karl,After most of her family is murdered in a terr...,"after family murdered terrorist bombing , youn..."
3,Jeans,When the father of the man she loves insists t...,when father loves insists twin marry twin sist...
4,Grown Ups,Mourning the loss of their beloved junior high...,mourning loss beloved junior high basketball c...


In [10]:
filtered_tv = []
tv_words = []

for text in tv_des['description']:
    text_tokens = word_tokenize(text)
    tokens_without_sw = [word.lower() for word in text_tokens if not word in stopwords.words()]
    tv_words.append(tokens_without_sw)
    filtered = (" ").join(tokens_without_sw)
    filtered_tv.append(filtered)

tv_words = [val for sublist in tv_words for val in sublist]
tv_words = sorted(set(tv_words))
tv_des['description_filtered'] = filtered_tv
tv_des.head()

,title,description,description_filtered
0,Blood & Water,"After crossing paths at a party, a Cape Town t...","after crossing paths party , cape town sets pr..."
1,Kota Factory,In a city of coaching centers known to train I...,in city coaching centers train india ’ finest ...
2,The Great British Baking Show,A talented batch of amateur bakers face off in...,a talented batch amateur bakers 10-week compet...
3,Dear White People,Students of color navigate the daily slights a...,students color navigate daily slights slippery...
4,Falsa identidad,Strangers Diego and Isabel flee their home in ...,strangers diego isabel flee home mexico preten...


In [11]:
movie_word_binary = [[0] * 0 for i in range(len(set(movies_words)))]

for des in movies_des['description_filtered']:
    k = 0
    for word in movies_words:
        if word in des:
            movie_word_binary[k].append(1.0)
        else:
            movie_word_binary[k].append(0.0)
        k+=1
        
movie_word_binary = pd.DataFrame(movie_word_binary).transpose()

In [12]:
tv_word_binary = [[0] * 0 for i in range(len(set(tv_words)))]

for des in tv_des['description_filtered']:
    k = 0
    for word in tv_words:
        if word in des:
            tv_word_binary[k].append(1.0)
        else:
            tv_word_binary[k].append(0.0)
        k+=1
        
tv_word_binary = pd.DataFrame(tv_word_binary).transpose()

In [13]:
def recommender2(search):
    cs_list = []
    binary_list = []
    if search in movies_des['title'].values:
        idx = movies_des[movies_des['title'] == search].index.item()
        for i in movie_word_binary.iloc[idx]:
            binary_list.append(i)
        point1 = np.array(binary_list).reshape(1, -1)
        point1 = [val for sublist in point1 for val in sublist]    
        for j in range(len(movies_des)):
            binary_list2 = []
            for k in movie_word_binary.iloc[j]:
                binary_list2.append(k)
            point2 = np.array(binary_list2).reshape(1, -1)
            point2 = [val for sublist in point2 for val in sublist]
            dot_product = np.dot(point1, point2)
            norm_1 = np.linalg.norm(point1)
            norm_2 = np.linalg.norm(point2)
            cos_sim = dot_product / (norm_1 * norm_2)
            cs_list.append(cos_sim)
        movies_copy = movies_des.copy()
        movies_copy['cos_sim'] = cs_list
        results = movies_copy.sort_values('cos_sim', ascending=False)
        results = results[results['title'] != search]    
        top_results = results.head(5)
        return(top_results)
    elif search in tv_des['title'].values:
        idx = tv_des[tv_des['title'] == search].index.item()
        for i in tv_word_binary.iloc[idx]:
            binary_list.append(i)
        point1 = np.array(binary_list).reshape(1, -1)
        point1 = [val for sublist in point1 for val in sublist]
        for j in range(len(tv)):
            binary_list2 = []
            for k in tv_word_binary.iloc[j]:
                binary_list2.append(k)
            point2 = np.array(binary_list2).reshape(1, -1)
            point2 = [val for sublist in point2 for val in sublist]
            dot_product = np.dot(point1, point2)
            norm_1 = np.linalg.norm(point1)
            norm_2 = np.linalg.norm(point2)
            cos_sim = dot_product / (norm_1 * norm_2)
            cs_list.append(cos_sim)
        tv_copy = tv_des.copy()
        tv_copy['cos_sim'] = cs_list
        results = tv_copy.sort_values('cos_sim', ascending=False)
        results = results[results['title'] != search]    
        top_results = results.head(5)
        return(top_results)
    else:
        return("Title not in dataset. Please check spelling.")

## Recommending Movies

In [14]:
pd.options.display.max_colwidth = 300
recommender2('The Conjuring')

,title,description,description_filtered,cos_sim
2872,Death Note,"Light Turner finds a supernatural notebook and uses it to mete out death, attracting the attention of a detective, a demon and a girl in his class.","light turner finds supernatural notebook mete death , attracting attention detective , demon girl class .",0.460283
2193,Revenger,"Hell-bent on avenging the murder of his family, a former detective infiltrates a remote island that serves as a prison for vicious death row criminals.","hell-bent avenging murder family , former detective infiltrates remote island serves prison vicious death row criminals .",0.453147
297,A Haunted House 2,"Grieving after a tragedy, Malcolm tries to start fresh with his new girlfriend and her kids — but supernatural shenanigans find their way back to him.","grieving tragedy , malcolm start fresh girlfriend kids — supernatural shenanigans find back .",0.451781
3276,Ég man þig,"Young urbanites renovating a rundown house, and a psychiatrist grieving his son's disappearance, are connected to a supernatural, decades-old secret.","young urbanites renovating rundown house , psychiatrist grieving 's disappearance , connected supernatural , decades-old secret .",0.445435
3900,Ghost House,"While visiting the site of a supernatural shrine in the Thai countryside, a vacationing couple unwittingly upsets a vengeful demon.","while visiting site supernatural shrine thai countryside , vacationing couple unwittingly upsets vengeful demon .",0.434046


In [15]:
recommender2("Child's Play")

,title,description,description_filtered,cos_sim
3927,Good People,A struggling couple can't believe their luck when they find a stash of money in the apartment of a neighbor who was recently murdered.,a struggling couple n't believe luck find stash money apartment neighbor recently murdered .,0.431373
5152,Trip to Bhangarh: Asia's Most Haunted Place,"To amuse themselves, six college friends decide to pay a visit to a fortress believed by some to be the most haunted place in Asia.","to amuse , college friends decide pay visit fortress believed haunted place asia .",0.420084
1358,Material,"A dutiful son must hide his pursuit of stand-up comedy from his staunch father, who expects him to inherit his store and uphold their Muslim beliefs.","a dutiful hide pursuit stand-up comedy staunch father , expects inherit store uphold muslim beliefs .",0.419219
209,Rehmataan,"As unemployment, drug addiction and corruption plague a society, this drama depicts the people who believe there’s still good in the world.","as unemployment , addiction corruption plague society , drama depicts believe ’ world .",0.417483
2009,"Extremely Wicked, Shockingly Evil and Vile",Single mother Liz falls for Ted Bundy and refuses to believe the truth about his crimes for years. A drama based on a true story.,single mother liz falls ted bundy refuses believe truth crimes years . a drama based story .,0.411665


In [16]:
recommender2('Charlie and the Chocolate Factory')

,title,description,description_filtered,cos_sim
5235,Willy Wonka & the Chocolate Factory,Zany Willy Wonka causes a stir when he announces that golden tickets hidden inside his candy bars will admit holders into his secret confectionary.,zany willy wonka stir announces golden tickets hidden inside candy bars admit holders secret confectionary .,0.422200
95,Kyaa Kool Hain Hum 3,"When an unlikely porn actor falls for a woman outside the industry, he employs his co-stars as a stand-in traditional family to impress her father.","when porn actor falls woman industry , employs co-stars stand-in traditional family impress father .",0.372742
1885,Kaake Da Viyah,"In this zany comedy, a man is torn between the girl he loves and the respective women his warring mother and grandmother have chosen for him to marry.","in zany comedy , torn girl loves respective women warring mother grandmother chosen marry .",0.355643
4205,Lechmi,"The spirit of a murdered woman contacts four bachelors, enlisting their help to find her killer.","the spirit murdered woman contacts bachelors , enlisting find killer .",0.353161
3234,BoJack Horseman Christmas Special: Sabrina's Christmas Wish,"It's Christmas, and BoJack wants nothing to do with it. Then Todd shows up with a giant candy cane and an old ""Horsin' Around"" Christmas episode.","it 's christmas , bojack . then todd shows giant candy cane `` horsin ' around '' christmas episode .",0.352034


In [17]:
recommender2('Wild Child')

,title,description,description_filtered,cos_sim
4852,The Blackcoat's Daughter,"When their parents fail to pick them up for winter break, two students stay on at an all-girls boarding school in the company of a terrible presence.","when parents fail pick winter break , students stay all-girls boarding school company terrible presence .",0.446910
4545,Princess Cyd,A teenager moves to Chicago for the summer and finds a poignant mentor in her aunt and a sweet love interest in a girl who works in the neighborhood.,a teenager moves chicago summer finds poignant mentor aunt sweet love interest girl works neighborhood .,0.429377
1912,Misfit,"After living in America for years, a teenage girl moves back to the Netherlands and is quickly singled out as a misfit by the popular clique at school.","after living america years , teenage girl moves back netherlands quickly singled misfit popular clique school .",0.427247
2394,The Black Prince,"After being exiled to England as a child, a Punjabi prince struggles to return to his homeland and his Sikh faith in this historical drama.","after exiled england child , punjabi prince struggles return homeland sikh faith historical drama .",0.425307
3971,He Named Me Malala,"This documentary tells the story of a teenage Pakistani girl shot for her advocacy of women's education, her survival, and her continued efforts.","this documentary tells story teenage pakistani girl shot advocacy women 's education , survival , continued efforts .",0.421350


In [18]:
recommender2("Dr. Seuss' The Cat in the Hat")

,title,description,description_filtered,cos_sim
1431,Dark Light,"Implicated in her daughter's disappearance, a mother searches for answers when she returns to an old family home that may have an unwanted visitor.","implicated daughter 's disappearance , mother searches answers returns family home unwanted visitor .",0.500343
3972,He's Out There,"While vacationing at a remote lake house, a mother and her daughters become pawns in the twisted game of an ax-wielding psychopath.","while vacationing remote lake house , mother daughters pawns twisted game ax-wielding psychopath .",0.476212
4862,The Breadwinner,A courageous 11-year-old Afghan girl disguises herself as a boy and takes on odd jobs to provide for her family when her father is arrested.,a courageous 11-year-old afghan girl disguises boy takes odd jobs provide family father arrested .,0.471782
4227,Little Sister,"A former Goth studying to become a nun finds her faith tested when she returns home to visit her brother, who has been disfigured in the Iraq War.","a former goth studying finds faith tested returns home visit brother , disfigured iraq war .",0.471782
4060,INDIA,"A man buys a young girl, code-names her ""Doll"" and sends her to live with a mother of two who has a mysterious mission to be fulfilled.","a buys young girl , code-names `` doll '' sends live mother mysterious mission fulfilled .",0.463281


In [21]:
recommender2('Holidate')

,title,description,description_filtered,cos_sim
5255,Yanda Kartavya Aahe,"Thanks to an arranged marriage that was designed to make an ailing grandmother happy, newlyweds Rahul and Swati are virtually strangers. Can a four-day honeymoon make a difference when it comes to the couple's feelings about each other?","thanks arranged marriage designed make ailing grandmother happy , newlyweds rahul swati virtually strangers . can four-day honeymoon make difference couple 's feelings ?",0.471041
4114,Just Friends,"Years after high school, a once-portly nice guy comes home for the holidays and reunites with a former crush he still has feelings for.","years high school , once-portly nice guy home holidays reunites former crush feelings .",0.464953
4575,Red Christmas,A family Christmas gathering at an isolated Australian estate turns into the holiday from hell after a mysterious stranger turns up at the door.,a family christmas gathering isolated australian estate turns holiday hell mysterious stranger turns .,0.461555
2810,The Hateful Eight,"Years after the Civil War, a bounty hunter and his captive are waylaid by a Wyoming blizzard and hole up in a way station with six dicey strangers.","years civil war , bounty hunter captive waylaid wyoming blizzard hole station dicey strangers .",0.448343
2580,Us and Them,"Two strangers meet on a train and form a bond that evolves over the years. After a separation, they reconnect and reflect on their love for each other.","two strangers meet train form bond evolves years . after separation , reconnect reflect love .",0.441876
